In [1]:
# import pandas
import pandas as pd

In [3]:
cars = pd.read_csv('../Data/car-cleaned-data.csv')
cars.head()

,name,company,year,Price,kms_driven,fuel_type
0,Hyundai Santro Xing,Hyundai,2007,80000.000,45000,Petrol
1,Mahindra Jeep CL550,Mahindra,2006,425000.000,40,Diesel
2,Maruti Suzuki Alto,Maruti,2018,411717.615,22000,Petrol
3,Hyundai Grand i10,Hyundai,2014,325000.000,28000,Petrol
4,Ford EcoSport Titanium,Ford,2014,575000.000,36000,Diesel


In [15]:
X = cars.drop(columns='Price')
y = cars['Price']

In [16]:
X

,name,company,year,kms_driven,fuel_type
0,Hyundai Santro Xing,Hyundai,2007,45000,Petrol
1,Mahindra Jeep CL550,Mahindra,2006,40,Diesel
2,Maruti Suzuki Alto,Maruti,2018,22000,Petrol
3,Hyundai Grand i10,Hyundai,2014,28000,Petrol
4,Ford EcoSport Titanium,Ford,2014,36000,Diesel
...,...,...,...,...,...
832,Maruti Suzuki Ritz,Maruti,2011,50000,Petrol
833,Tata Indica V2,Tata,2009,30000,Diesel
834,Toyota Corolla Altis,Toyota,2009,132000,Petrol
835,Tata Zest XM,Tata,2018,27000,Diesel


In [8]:
# import test train split
from sklearn.model_selection import train_test_split

In [9]:
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.2,random_state=101)

In [13]:
# make some necessary imports
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder

In [21]:
ohe=OneHotEncoder()
ohe.fit(X[['name','company','fuel_type']])

OneHotEncoder()

In [35]:
# ohe.categories_

In [23]:
# make column transformer to encode the columns
ctohe = ColumnTransformer(
    transformers=[("ohe", OneHotEncoder(categories=ohe.categories_,sparse=False,drop='first'), [0,1,4])],
    remainder='passthrough')

lr = LinearRegression()

pipe = Pipeline([('ctohe',ctohe),('lr',lr)])
pipe.fit(X_train,y_train)
y_preds = pipe.predict(X_test)

In [26]:
from sklearn.metrics import r2_score,mean_absolute_error

In [28]:
print(f"The R2 Score is {r2_score(y_test,y_preds)}")

The R2 Score is 0.5023087415137741


In [29]:
# Check for better model random state
scores=[]
for i in range(1000):
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=i)
    lr=LinearRegression()
    pipe = Pipeline([('ctohe',ctohe),('lr',lr)])
    pipe.fit(X_train,y_train)
    y_pred=pipe.predict(X_test)
    scores.append(r2_score(y_test,y_pred))

In [30]:
import numpy as np

In [31]:
np.argmax(scores)

995

In [32]:
scores[np.argmax(scores)]

0.8976635097032695

# The Best Model Found at a random state

In [33]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=np.argmax(scores))
lr=LinearRegression()
pipe = Pipeline([('ctohe',ctohe),('lr',lr)])
pipe.fit(X_train,y_train)
y_pred=pipe.predict(X_test)
r2_score(y_test,y_pred)

0.8976635097032695

In [34]:
pipe.predict(pd.DataFrame(columns=['name','company','year','kms_driven','fuel_type'],data=np.array(['Maruti Suzuki Swift','Maruti',2019,100,'Petrol']).reshape(1,5)))

array([427437.35565783])

In [36]:
import pickle

In [38]:
pickle.dump(pipe,open('../Model/pipe.pkl','wb'))